In [1]:
import numpy as np
import pandas as pd
from itertools import permutations, combinations_with_replacement

In [2]:
import spacy

In [3]:
from experiments import CHAR_VOCAB_PATH
from utils import generate_vocab_mappings, load_sentences

In [4]:
itos, stoi = generate_vocab_mappings(CHAR_VOCAB_PATH)

In [5]:
sentences_w_number = load_sentences("input_sentences/refactored_sentences.txt")

In [6]:
nlp = spacy.load('de_core_news_sm')

### Generating german dictionary 

In [7]:
def generate_german_dict(path):
    # TODO: path is "vocabularies/de_dict.csv"
    df = pd.read_csv(path)
    df = df[['lemma','genus','akkusativ singular','akkusativ singular 1',
             'dativ singular','dativ singular 1','genitiv singular','genitiv singular 1',
             'nominativ plural','nominativ plural 1','akkusativ plural','akkusativ plural 1',
             'dativ plural','dativ plural 1','genitiv plural','genitiv plural 1']]
    df = df.rename(columns={
                                 'lemma':'nom sg',
                                 'akkusativ singular':'acc sg',
                                 'akkusativ singular 1':'acc1 sg',
                                 'dativ singular':'dat sg',
                                 'dativ singular 1':'dat1 sg',
                                 'genitiv singular':'gen sg',
                                 'genitiv singular 1':'gen1 sg',
                                 'nominativ plural':'nom pl',
                                 'nominativ plural 1':'nom1 pl',
                                 'akkusativ plural':'acc pl',
                                 'akkusativ plural 1':'acc1 pl',
                                 'dativ plural':'dat pl',
                                 'dativ plural 1':'dat1 pl',
                                 'genitiv plural':'gen pl',
                                 'genitiv plural 1':'gen1 pl'
                                 })
    
    df = df.replace(np.nan, '', regex=True)
    df['acc sg'] = df['acc sg'] + df['acc1 sg']
    df['dat sg'] = df['dat sg'] + df['dat1 sg']
    df['gen sg'] = df['gen sg'] + df['gen1 sg']
    df['nom pl'] = df['nom pl'] + df['nom1 pl']
    df['acc pl'] = df['acc pl'] + df['acc1 pl']
    df['dat pl'] = df['dat pl'] + df['dat1 pl']
    df['gen pl'] = df['gen pl'] + df['gen1 pl']
    df = df.drop(['acc1 sg','dat1 sg','gen1 sg','nom1 pl','acc1 pl','dat1 pl','gen1 pl'], axis=1)
    #duplicates = df.duplicated(subset=None, keep='first')
    print(df.shape)
    german_dict = df.set_index('nom sg').T.to_dict('list')
    return german_dict, df

In [8]:
german_dict, df = generate_german_dict("vocabularies/de_dict.csv")

/Users/charlotterochereau/anaconda3/envs/char_lm_fb/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3209: DtypeWarning: Columns (5,9,18,27,36,45,54,63,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


(79198, 9)


/Users/charlotterochereau/anaconda3/envs/char_lm_fb/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: DataFrame columns are not unique, some columns will be omitted.


In [23]:
print(stoi["redner"])

12823


In [24]:
# testing german_dict
print(german_dict["Redner"])

['m', 'Redner', 'Redner', 'Redners', 'Redner', 'Redner', 'Rednern', 'Redner']


### Parsing using spacy

In [10]:
print(sentences_w_number)

Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat. Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat. Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat. Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat. Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat. Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat. Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat. Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat. Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat. Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat. Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat. Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat. Wir erfahren, dass die Presse sg der Z

In [11]:
sentences_wo_number = sentences_w_number.replace('sg ','').replace('pl ','')

In [12]:
print(sentences_wo_number)

Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat. Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat. Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat. Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat. Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat. Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat. Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat. Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat. Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat. Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat. Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat. Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat. Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat. Sie hören, dass der Senat der Abgeordneter eine Maßnahme gemeldet hat. Ich 

In [13]:
doc_w_number = nlp(sentences_w_number)
print("doc with number:",len(doc_w_number))
doc_wo_number = nlp(sentences_wo_number)
print("doc without number:",len(doc_wo_number))

doc with number: 320
doc without number: 260


### Sentence segmenter

In [14]:
list_sentences_w_number = []
for sentence in doc_w_number.sents:
    list_sentences_w_number.append(sentence.text)
print(list_sentences_w_number)

['Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat.', 'Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat.', 'Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat.', 'Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat.', 'Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat.', 'Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat.', 'Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat.', 'Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat.', 'Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat.', 'Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat.', 'Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat.', 'Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat.', '

In [15]:
list_sentences_wo_number = []
for sentence in doc_wo_number.sents:
    list_sentences_wo_number.append(sentence.text)
print(list_sentences_wo_number)

['Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat.', 'Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat.', 'Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat.', 'Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat.', 'Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat.', 'Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat.', 'Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat.', 'Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat.', 'Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat.', 'Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat.', 'Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat.', 'Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat.', 'Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat.', 'Sie hören, dass der Senat der Abge

In [16]:
assert len(list_sentences_w_number) == len(list_sentences_wo_number)
print("number of sentences:",len(list_sentences_w_number))
print("characters in sentences with number:", len(list_sentences_w_number[0]))
print("characters in sentences without number:", len(list_sentences_wo_number[0]))

number of sentences: 20
characters in sentences with number: 70
characters in sentences without number: 61


In [17]:
number_sentences = len(list_sentences_wo_number)
print(number_sentences)

20


### Tokenizing using spacy

In [18]:
doc_list_w_number = []
for sentence in range(number_sentences):
    doc_list_w_number.append(nlp(list_sentences_w_number[sentence]))

In [19]:
print(doc_list_w_number)

[Wir glauben, dass der Mann sg die Frau pl ein Geschenk sg gegeben hat., Wir wissen, dass der Bankier sg sein Kunde pl eine Finanzierung sg angeboten hat., Wir wissen, dass das Kindermädchen sg das Kind sg ein Buch sg gekauft hat., Wir vermuten, dass die Hausherrin sg sein Gast pl ein Wein sg angebietet hat., Wir denken, dass der Mann sg sein Freund pl eine Sehenswürdigkeit pl gezeigt hat., Wir bezweifeln, dass der Verdächtiger sg sein Anwalt sg die Wahrheit sg verheimlicht hat., Wir denken, dass der Arzt sg der Patient sg eine Behandlung sg verschieben hat., Wir erfahren, dass der Eigentümer sg ein Student sg seine Wohnung sg vermietet hat., Wir verdächtigen, dass der Spion sg ihr Feind sg das Geheimnis pl verratet hat., Wir denken, dass die Frau sg sein Partner sg eine Überraschung sg versprochen hat., Ihr weisst, dass die Mutter sg ihre Tochter sg ein Paket sg geschickt hat., Sie sagen, dass der Sportler sg seine Nachbarin sg sein Fahrrad sg verliehen hat., Wir erfahren, dass die Pr

In [20]:
tokens_w_number = [[] for _ in range(number_sentences)]
for idx_sentence, sentence in enumerate(doc_list_w_number):
    for token in sentence: 
        tokens_w_number[idx_sentence].append(token.text)

In [21]:
doc_list_wo_number = []
for sentence in range(number_sentences):
    doc_list_wo_number.append(nlp(list_sentences_wo_number[sentence]))

In [22]:
print(doc_list_wo_number)

[Wir glauben, dass der Mann die Frau ein Geschenk gegeben hat., Wir wissen, dass der Bankier sein Kunde eine Finanzierung angeboten hat., Wir wissen, dass das Kindermädchen das Kind ein Buch gekauft hat., Wir vermuten, dass die Hausherrin sein Gast ein Wein angebietet hat., Wir denken, dass der Mann sein Freund eine Sehenswürdigkeit gezeigt hat., Wir bezweifeln, dass der Verdächtiger sein Anwalt die Wahrheit verheimlicht hat., Wir denken, dass der Arzt der Patient eine Behandlung verschieben hat., Wir erfahren, dass der Eigentümer ein Student seine Wohnung vermietet hat., Wir verdächtigen, dass der Spion ihr Feind das Geheimnis verratet hat., Wir denken, dass die Frau sein Partner eine Überraschung versprochen hat., Ihr weisst, dass die Mutter ihre Tochter ein Paket geschickt hat., Sie sagen, dass der Sportler seine Nachbarin sein Fahrrad verliehen hat., Wir erfahren, dass die Presse der Zuhörer die Nachricht verkündet hat., Sie hören, dass der Senat der Abgeordneter eine Maßnahme geme

In [23]:
tokens_wo_number = [[] for _ in range(number_sentences)]
for idx_sentence, sentence in enumerate(doc_list_wo_number):
    for token in sentence: 
        tokens_wo_number[idx_sentence].append(token.text)

In [24]:
print(tokens_wo_number)

[['Wir', 'glauben', ',', 'dass', 'der', 'Mann', 'die', 'Frau', 'ein', 'Geschenk', 'gegeben', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'der', 'Bankier', 'sein', 'Kunde', 'eine', 'Finanzierung', 'angeboten', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'das', 'Kindermädchen', 'das', 'Kind', 'ein', 'Buch', 'gekauft', 'hat', '.'], ['Wir', 'vermuten', ',', 'dass', 'die', 'Hausherrin', 'sein', 'Gast', 'ein', 'Wein', 'angebietet', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Mann', 'sein', 'Freund', 'eine', 'Sehenswürdigkeit', 'gezeigt', 'hat', '.'], ['Wir', 'bezweifeln', ',', 'dass', 'der', 'Verdächtiger', 'sein', 'Anwalt', 'die', 'Wahrheit', 'verheimlicht', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Arzt', 'der', 'Patient', 'eine', 'Behandlung', 'verschieben', 'hat', '.'], ['Wir', 'erfahren', ',', 'dass', 'der', 'Eigentümer', 'ein', 'Student', 'seine', 'Wohnung', 'vermietet', 'hat', '.'], ['Wir', 'verdächtigen', ',', 'dass', 'der', 'Spion', 'ihr', 'Feind', 'das', 'Geheimni

In [25]:
print(tokens_wo_number)

[['Wir', 'glauben', ',', 'dass', 'der', 'Mann', 'die', 'Frau', 'ein', 'Geschenk', 'gegeben', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'der', 'Bankier', 'sein', 'Kunde', 'eine', 'Finanzierung', 'angeboten', 'hat', '.'], ['Wir', 'wissen', ',', 'dass', 'das', 'Kindermädchen', 'das', 'Kind', 'ein', 'Buch', 'gekauft', 'hat', '.'], ['Wir', 'vermuten', ',', 'dass', 'die', 'Hausherrin', 'sein', 'Gast', 'ein', 'Wein', 'angebietet', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Mann', 'sein', 'Freund', 'eine', 'Sehenswürdigkeit', 'gezeigt', 'hat', '.'], ['Wir', 'bezweifeln', ',', 'dass', 'der', 'Verdächtiger', 'sein', 'Anwalt', 'die', 'Wahrheit', 'verheimlicht', 'hat', '.'], ['Wir', 'denken', ',', 'dass', 'der', 'Arzt', 'der', 'Patient', 'eine', 'Behandlung', 'verschieben', 'hat', '.'], ['Wir', 'erfahren', ',', 'dass', 'der', 'Eigentümer', 'ein', 'Student', 'seine', 'Wohnung', 'vermietet', 'hat', '.'], ['Wir', 'verdächtigen', ',', 'dass', 'der', 'Spion', 'ihr', 'Feind', 'das', 'Geheimni

In [26]:
print("sentences in tokens", len(tokens_w_number))
print("characters in tokens with number:", len(tokens_w_number[0]))
print("characters in tokens without number:", len(tokens_wo_number[0]))

sentences in tokens 20
characters in tokens with number: 16
characters in tokens without number: 13


### Selecting verb arguments

In [27]:
# Backup
def select_args_wo_nb(args_nb, nb_sentences, tokens):

    verb_args = [[] for _ in range(nb_sentences)]
    for sentence_idx, sentence in enumerate(tokens):
        verb_args[sentence_idx].append(tokens[sentence_idx][4] + " " + tokens[sentence_idx][5])
        verb_args[sentence_idx].append(tokens[sentence_idx][6] + " " + tokens[sentence_idx][7])
        verb_args[sentence_idx].append(tokens[sentence_idx][8] + " " + tokens[sentence_idx][9])

    return verb_args

In [28]:
args_wo_number = [[] for _ in range(number_sentences)]
for sentence_idx, sentence in enumerate(tokens_wo_number):
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][4]+" "+tokens_wo_number[sentence_idx][5])
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][6]+" "+tokens_wo_number[sentence_idx][7])
    args_wo_number[sentence_idx].append(tokens_wo_number[sentence_idx][8]+" "+tokens_wo_number[sentence_idx][9])
print(args_wo_number)

[['der Mann', 'die Frau', 'ein Geschenk'], ['der Bankier', 'sein Kunde', 'eine Finanzierung'], ['das Kindermädchen', 'das Kind', 'ein Buch'], ['die Hausherrin', 'sein Gast', 'ein Wein'], ['der Mann', 'sein Freund', 'eine Sehenswürdigkeit'], ['der Verdächtiger', 'sein Anwalt', 'die Wahrheit'], ['der Arzt', 'der Patient', 'eine Behandlung'], ['der Eigentümer', 'ein Student', 'seine Wohnung'], ['der Spion', 'ihr Feind', 'das Geheimnis'], ['die Frau', 'sein Partner', 'eine Überraschung'], ['die Mutter', 'ihre Tochter', 'ein Paket'], ['der Sportler', 'seine Nachbarin', 'sein Fahrrad'], ['die Presse', 'der Zuhörer', 'die Nachricht'], ['der Senat', 'der Abgeordneter', 'eine Maßnahme'], ['der Junge', 'sein Kumpel', 'ein Kugelschreiber'], ['das Theater', 'der Musiker', 'ein Sitzplatz'], ['der Schüler', 'der Lehrer', 'ihre Frage'], ['der Professor', 'der Student', 'ein Fach'], ['der Chef', 'das Kind', 'ein Kuchen'], ['der Priester', 'der Gläubiger', 'ihr Fehler']]


In [29]:
args_number = 3
args_w_number = [[[] for _ in range(args_number)] for _ in range(number_sentences)]
for sentence_idx, sentence in enumerate(tokens_w_number):
    args_w_number[sentence_idx][0]=(tokens_w_number[sentence_idx][4:7])
    args_w_number[sentence_idx][1]=(tokens_w_number[sentence_idx][7:10])
    args_w_number[sentence_idx][2]=(tokens_w_number[sentence_idx][10:13])
print(args_w_number)

[[['der', 'Mann', 'sg'], ['die', 'Frau', 'pl'], ['ein', 'Geschenk', 'sg']], [['der', 'Bankier', 'sg'], ['sein', 'Kunde', 'pl'], ['eine', 'Finanzierung', 'sg']], [['das', 'Kindermädchen', 'sg'], ['das', 'Kind', 'sg'], ['ein', 'Buch', 'sg']], [['die', 'Hausherrin', 'sg'], ['sein', 'Gast', 'pl'], ['ein', 'Wein', 'sg']], [['der', 'Mann', 'sg'], ['sein', 'Freund', 'pl'], ['eine', 'Sehenswürdigkeit', 'pl']], [['der', 'Verdächtiger', 'sg'], ['sein', 'Anwalt', 'sg'], ['die', 'Wahrheit', 'sg']], [['der', 'Arzt', 'sg'], ['der', 'Patient', 'sg'], ['eine', 'Behandlung', 'sg']], [['der', 'Eigentümer', 'sg'], ['ein', 'Student', 'sg'], ['seine', 'Wohnung', 'sg']], [['der', 'Spion', 'sg'], ['ihr', 'Feind', 'sg'], ['das', 'Geheimnis', 'pl']], [['die', 'Frau', 'sg'], ['sein', 'Partner', 'sg'], ['eine', 'Überraschung', 'sg']], [['die', 'Mutter', 'sg'], ['ihre', 'Tochter', 'sg'], ['ein', 'Paket', 'sg']], [['der', 'Sportler', 'sg'], ['seine', 'Nachbarin', 'sg'], ['sein', 'Fahrrad', 'sg']], [['die', 'Presse

In [30]:
print(len(args_w_number))
print(len(args_w_number[0]))

20
3


### Generate case permutations

In [31]:
def get_nom(group_args):
    # get nominative of verb argument and corresponding article
    new_nom = []
    article = group_args[0]
    word = group_args[1]
    number = group_args[2]
    
    if number == "sg":
        new_nom.append(article+" "+word)

    elif number == "pl":
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "die"
        elif article in {"sein","seine"}:
            pl_article = "seine"
        elif article in {"ihr","ihre"}:
            pl_article = "ihre"
        elif article in {"ein","eine"}:
            pl_article = "einige"
        
        pl_word = german_dict[word][4]
        new_nom.append(pl_article+" "+pl_word)
        
    return new_nom

In [32]:
print(get_nom(args_w_number[0][0]))
print(get_nom(args_w_number[0][1]))
print(get_nom(args_w_number[17][2]))
print(get_nom(args_w_number[18][2]))

['der Mann']
['die Frauen']
['einige Fächer']
['ein Kuchen']


In [33]:
def get_acc(group_args):
    # get accusative of verb argument and corresponding article
    new_acc = []
    article = group_args[0]
    word = group_args[1]
    gender = german_dict[word][0]
    number = group_args[2]

    if number == "sg":
        sg_article = ""
        sg_word = ""
        if gender == "m":
            if article == "der":
                sg_article = "den"
            elif article == "sein":
                sg_article = "seinen"
            elif article == "ihr":
                sg_article = "ihren"
            else:
                sg_article = "einen"
        else:
            sg_article = article
        
        sg_word = german_dict[word][1]
        
        new_acc.append(sg_article+" "+sg_word)
        
    elif number == "pl":
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "die"
        elif article in {"sein","seine"}:
            pl_article = "seine"
        elif article in {"ihr","ihre"}:
            pl_article = "ihre"
        elif article in {"ein","eine"}:
            pl_article = "einige"
            
        pl_word = german_dict[word][5]   
        new_acc.append(pl_article+" "+pl_word)
        
    return new_acc

In [34]:
print(get_acc(args_w_number[0][0]))
print(get_acc(args_w_number[0][1]))
print(get_acc(args_w_number[1][0]))
print(get_acc(args_w_number[17][2]))

['den Mann']
['die Frauen']
['den Bankier']
['einige Fächer']


In [35]:
def get_dat(group_args):
    # get dative of verb argument and corresponding article
    new_dat = []
    article = group_args[0]
    word = group_args[1]
    gender = german_dict[word][0]
    number = group_args[2]
    
    if number == "sg":
        sg_article = ""
        sg_word = ""
        if gender in {"m","n"}:
            if article in {"der","das"}:
                sg_article = "dem"
            elif article == "sein":
                sg_article = "seinem"
            elif article == "ihr":
                sg_article = "ihrem"
            elif article == "ein":
                sg_article = "einem"
        elif gender == "f":
            if article == "die":
                sg_article = "der"
            elif article == "seine":
                sg_article = "seiner"
            elif article == "ihre":
                sg_article = "ihrer"
            elif article == "eine":
                sg_article = "einer"
        
        sg_word = german_dict[word][2] 
        new_dat.append(sg_article+" "+sg_word)
        
    elif number == "pl": 
        pl_article = ""
        pl_word = ""
        if article in {"der","die","das"}:
            pl_article = "den"
        elif article in {"sein","seine"}:
            pl_article = "seinen"
        elif article in {"ihr","ihre"}:
            pl_article = "ihren" 
        elif article in {"ein","eine"}:
            pl_article = "einigen"
    
        pl_word = german_dict[word][6]     
        new_dat.append(pl_article+" "+pl_word)
        
    return new_dat

In [36]:
print(get_dat(args_w_number[0][0]))
print(get_dat(args_w_number[0][1]))
print(get_dat(args_w_number[17][0]))
print(get_dat(args_w_number[17][1]))
print(get_dat(args_w_number[17][2]))
print(get_nom(args_w_number[18][2]))

['dem Mann']
['den Frauen']
['dem Professor']
['den Studenten']
['einigen Fächern']
['ein Kuchen']


In [37]:
def get_all_cases(list_args, number_args):
    all_cases = [[[] for _ in range(number_args)] for _ in range(number_sentences)]
    for sentence_idx, sentence in enumerate(args_w_number):
        for group_args_idx, group_args in enumerate(args_w_number[sentence_idx]):
            nom = get_nom(args_w_number[sentence_idx][group_args_idx])
            acc = get_acc(args_w_number[sentence_idx][group_args_idx])
            dat = get_dat(args_w_number[sentence_idx][group_args_idx])
            all_cases[sentence_idx][group_args_idx].append(nom)
            all_cases[sentence_idx][group_args_idx].append(acc)
            all_cases[sentence_idx][group_args_idx].append(dat)
    return all_cases

In [38]:
case_permutations = get_all_cases(args_w_number, 3)
print(len(case_permutations))
print(len(case_permutations[0]))
print(case_permutations)

20
3
[[[['der Mann'], ['den Mann'], ['dem Mann']], [['die Frauen'], ['die Frauen'], ['den Frauen']], [['ein Geschenk'], ['ein Geschenk'], ['einem Geschenk']]], [[['der Bankier'], ['den Bankier'], ['dem Bankier']], [['seine Kunden'], ['seine Kunden'], ['seinen Kunden']], [['eine Finanzierung'], ['eine Finanzierung'], ['einer Finanzierung']]], [[['das Kindermädchen'], ['das Kindermädchen'], ['dem Kindermädchen']], [['das Kind'], ['das Kind'], ['dem Kind']], [['ein Buch'], ['ein Buch'], ['einem Buch']]], [[['die Hausherrin'], ['die Hausherrin'], ['der Hausherrin']], [['seine Gasten'], ['seine Gasten'], ['seinen Gasten']], [['ein Wein'], ['einen Wein'], ['einem Wein']]], [[['der Mann'], ['den Mann'], ['dem Mann']], [['seine Freunde'], ['seine Freunde'], ['seinen Freunden']], [['einige Sehenswürdigkeiten'], ['einige Sehenswürdigkeiten'], ['einigen Sehenswürdigkeiten']]], [[['der Verdächtiger'], ['den Verdächtigen'], ['dem Verdächtigem']], [['sein Anwalt'], ['seinen Anwalt'], ['seinem Anwalt

In [39]:
# 0 = Nom, 1 = Acc, 2 = Dat
idxs = [0,1,2]
perm = list(permutations(idxs))
print(perm)

[(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]


In [40]:
combinations = [[[] for _ in range(6)] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(case_permutations):
    for idx in range(6):
        for sub_idx in range(3):
            perm_idx = perm[idx][sub_idx]
            combinations[stc_idx][idx].append(case_permutations[stc_idx][sub_idx][perm_idx])

In [41]:
print(len(combinations))
print(len(combinations[0]))
print(combinations[0])

20
6
[[['der Mann'], ['die Frauen'], ['einem Geschenk']], [['der Mann'], ['den Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['einem Geschenk']], [['den Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']]]


In [42]:
print(combinations)

[[[['der Mann'], ['die Frauen'], ['einem Geschenk']], [['der Mann'], ['den Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['einem Geschenk']], [['den Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']]], [[['der Bankier'], ['seine Kunden'], ['einer Finanzierung']], [['der Bankier'], ['seinen Kunden'], ['eine Finanzierung']], [['den Bankier'], ['seine Kunden'], ['einer Finanzierung']], [['den Bankier'], ['seinen Kunden'], ['eine Finanzierung']], [['dem Bankier'], ['seine Kunden'], ['eine Finanzierung']], [['dem Bankier'], ['seine Kunden'], ['eine Finanzierung']]], [[['das Kindermädchen'], ['das Kind'], ['einem Buch']], [['das Kindermädchen'], ['dem Kind'], ['ein Buch']], [['das Kindermädchen'], ['das Kind'], ['einem Buch']], [['das Kindermädchen'], ['dem Kind'], ['ein Buch']], [['dem Kindermädchen'], ['das Kind'], ['ein Buch']], [['dem Kindermädchen'], ['das Kind'], ['ein Buch']]], [[[

In [43]:
all_perm = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(combinations):
    for idx in range(6):
        tmp_perm = list(permutations(combinations[stc_idx][idx]))
        all_perm[stc_idx].append(tmp_perm)

In [44]:
print(all_perm)

[[[(['der Mann'], ['die Frauen'], ['einem Geschenk']), (['der Mann'], ['einem Geschenk'], ['die Frauen']), (['die Frauen'], ['der Mann'], ['einem Geschenk']), (['die Frauen'], ['einem Geschenk'], ['der Mann']), (['einem Geschenk'], ['der Mann'], ['die Frauen']), (['einem Geschenk'], ['die Frauen'], ['der Mann'])], [(['der Mann'], ['den Frauen'], ['ein Geschenk']), (['der Mann'], ['ein Geschenk'], ['den Frauen']), (['den Frauen'], ['der Mann'], ['ein Geschenk']), (['den Frauen'], ['ein Geschenk'], ['der Mann']), (['ein Geschenk'], ['der Mann'], ['den Frauen']), (['ein Geschenk'], ['den Frauen'], ['der Mann'])], [(['den Mann'], ['die Frauen'], ['einem Geschenk']), (['den Mann'], ['einem Geschenk'], ['die Frauen']), (['die Frauen'], ['den Mann'], ['einem Geschenk']), (['die Frauen'], ['einem Geschenk'], ['den Mann']), (['einem Geschenk'], ['den Mann'], ['die Frauen']), (['einem Geschenk'], ['die Frauen'], ['den Mann'])], [(['den Mann'], ['den Frauen'], ['ein Geschenk']), (['den Mann'], ['

In [45]:
print(len(all_perm))
print(len(all_perm[0]))
print(len(all_perm[0][0]))
print(len(all_perm[0][0][0]))

20
6
6
3


In [46]:
#print(all_perm[0])
#print(all_perm[0][0])
print(all_perm[0][0][0])

(['der Mann'], ['die Frauen'], ['einem Geschenk'])


### Generate grammatical sentences

In [83]:
all_gramm = ""
all_gramm_list = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(tokens_wo_number): 
    for case_perm_idx, case_perm in enumerate(all_perm[stc_idx]):
        for case_idx, case in enumerate(all_perm[stc_idx][case_perm_idx]):
            principal_clause = f"{tokens_wo_number[stc_idx][0]} {tokens_wo_number[stc_idx][1]}, {tokens_wo_number[stc_idx][3]}"
            
            verb_arg = ""
            for vb_arg_idx, vb_arg in enumerate(all_perm[stc_idx][case_perm_idx][case_idx]):
                verb_arg += f"{vb_arg[0]} " 

            verb_subordinate = f"{tokens_wo_number[stc_idx][-3]} {tokens_wo_number[stc_idx][-2]}"
            all_gramm += f"{principal_clause} {verb_arg}{verb_subordinate}.\n"
            all_gramm_list[stc_idx].append(f"{principal_clause} {verb_arg}{verb_subordinate}.")
        
print(len(all_gramm))


52488


In [71]:
print(all_gramm)

Wir glauben, dass der Mann die Frauen einem Geschenk gegeben hat.
Wir glauben, dass der Mann einem Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen der Mann einem Geschenk gegeben hat.
Wir glauben, dass die Frauen einem Geschenk der Mann gegeben hat.
Wir glauben, dass einem Geschenk der Mann die Frauen gegeben hat.
Wir glauben, dass einem Geschenk die Frauen der Mann gegeben hat.
Wir glauben, dass der Mann den Frauen ein Geschenk gegeben hat.
Wir glauben, dass der Mann ein Geschenk den Frauen gegeben hat.
Wir glauben, dass den Frauen der Mann ein Geschenk gegeben hat.
Wir glauben, dass den Frauen ein Geschenk der Mann gegeben hat.
Wir glauben, dass ein Geschenk der Mann den Frauen gegeben hat.
Wir glauben, dass ein Geschenk den Frauen der Mann gegeben hat.
Wir glauben, dass den Mann die Frauen einem Geschenk gegeben hat.
Wir glauben, dass den Mann einem Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen den Mann einem Geschenk gegeben hat.
Wir glauben, dass die 

In [84]:
print(len(all_gramm_list))
print(len(all_gramm_list[0]))

20
36


In [49]:
BASE_PATH = "input_sentences/"
output_file = open(BASE_PATH + 'grammatical_sentences.txt', 'w')
output_file.write(all_gramm)
output_file.close()

### Generate ungrammatical sentences

In [50]:
# 0 = Nom, 1 = Acc, 2 = Dat
nb_cases = [0,1,2]
nb_case_repeats = list(combinations_with_replacement(nb_cases,3))
print(len(nb_case_repeats),[x for x in nb_case_repeats])

10 [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 1), (0, 1, 2), (0, 2, 2), (1, 1, 1), (1, 1, 2), (1, 2, 2), (2, 2, 2)]


In [51]:
for idx, value in enumerate(nb_case_repeats):
    if value in {(0,1,2),(0,0,0),(1,1,1),(2,2,2)}:
        nb_case_repeats.remove(value)
print(len(nb_case_repeats),nb_case_repeats)

6 [(0, 0, 1), (0, 0, 2), (0, 1, 1), (0, 2, 2), (1, 1, 2), (1, 2, 2)]


In [52]:
idxs_perms = [[] for _ in range(len(nb_case_repeats))]
for idx, value in enumerate(nb_case_repeats):
    tmp_idx_perm = list(permutations(value))
    idxs_perms[idx] = tmp_idx_perm
print(idxs_perms)
print(len(idxs_perms))

[[(0, 0, 1), (0, 1, 0), (0, 0, 1), (0, 1, 0), (1, 0, 0), (1, 0, 0)], [(0, 0, 2), (0, 2, 0), (0, 0, 2), (0, 2, 0), (2, 0, 0), (2, 0, 0)], [(0, 1, 1), (0, 1, 1), (1, 0, 1), (1, 1, 0), (1, 0, 1), (1, 1, 0)], [(0, 2, 2), (0, 2, 2), (2, 0, 2), (2, 2, 0), (2, 0, 2), (2, 2, 0)], [(1, 1, 2), (1, 2, 1), (1, 1, 2), (1, 2, 1), (2, 1, 1), (2, 1, 1)], [(1, 2, 2), (1, 2, 2), (2, 1, 2), (2, 2, 1), (2, 1, 2), (2, 2, 1)]]
6


In [53]:
"""
ungram_idxs = []
for idx, value in enumerate(idxs_perms):
    for i in set(idxs_perms[idx]):
        ungram_idxs.append(i)
print(ungram_idxs)
print(len(ungram_idxs))
"""

[(1, 0, 0), (0, 1, 0), (0, 0, 1), (0, 2, 0), (2, 0, 0), (0, 0, 2), (0, 1, 1), (1, 1, 0), (1, 0, 1), (2, 2, 0), (2, 0, 2), (0, 2, 2), (1, 2, 1), (2, 1, 1), (1, 1, 2), (1, 2, 2), (2, 2, 1), (2, 1, 2)]
18


In [163]:
ungram_idxs = [(1, 0, 0), (0, 1, 0), (0, 0, 1), (0, 2, 0), (2, 0, 0), (0, 0, 2), (0, 1, 1), (1, 1, 0), (1, 0, 1),(1, 2, 1), (2, 1, 1), (1, 1, 2),(2, 2, 0), (2, 0, 2), (0, 2, 2),  (1, 2, 2), (2, 2, 1), (2, 1, 2)]


In [54]:
print(len(case_permutations))
print(len(case_permutations[0]))

20
3


In [164]:
ungram_combinations = [[[]for _ in range(len(ungram_idxs))] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(case_permutations):
    for case_violation_idx, case_violation in enumerate(ungram_idxs):
        for sub_idx in range(3):
            perm_idx = ungram_idxs[case_violation_idx][sub_idx]
            ungram_combinations[stc_idx][case_violation_idx].append(case_permutations[stc_idx][sub_idx][perm_idx])

print('sentences:',len(ungram_combinations))
print('cases combinations:',len(ungram_combinations[0]))
print('permutations of each cases combination:',len(ungram_combinations[0][0]))
print(ungram_combinations[0])

sentences: 20
cases combinations: 18
permutations of each cases combination: 3
[[['den Mann'], ['die Frauen'], ['ein Geschenk']], [['der Mann'], ['die Frauen'], ['ein Geschenk']], [['der Mann'], ['die Frauen'], ['ein Geschenk']], [['der Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']], [['der Mann'], ['die Frauen'], ['einem Geschenk']], [['der Mann'], ['die Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['ein Geschenk']], [['den Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['ein Geschenk']], [['den Mann'], ['die Frauen'], ['einem Geschenk']], [['dem Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['einem Geschenk']], [['der Mann'], ['den Frauen'], ['einem Geschenk']], [['den Mann'], ['den Frauen'], ['einem Geschenk']], [['dem Mann'], ['den Frauen'], ['ein Geschenk']], [['dem Mann'], ['die Frauen'], ['einem Geschenk']]]


In [165]:
all_ungram_perm = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(ungram_combinations):
    for cases_idx, cases in enumerate(ungram_combinations[stc_idx]):
        tmp_perm = list(permutations(ungram_combinations[stc_idx][cases_idx]))
        all_ungram_perm[stc_idx].append(tmp_perm)
print('sentences:',len(all_ungram_perm))
print('cases combinations:',len(all_ungram_perm[0]))
print('permutations of positions:',len(all_ungram_perm[0][0]))
print(all_ungram_perm[0][0])
print(all_ungram_perm[18][1])

sentences: 20
cases combinations: 18
permutations of positions: 6
[(['den Mann'], ['die Frauen'], ['ein Geschenk']), (['den Mann'], ['ein Geschenk'], ['die Frauen']), (['die Frauen'], ['den Mann'], ['ein Geschenk']), (['die Frauen'], ['ein Geschenk'], ['den Mann']), (['ein Geschenk'], ['den Mann'], ['die Frauen']), (['ein Geschenk'], ['die Frauen'], ['den Mann'])]
[(['der Chef'], ['die Kinder'], ['ein Kuchen']), (['der Chef'], ['ein Kuchen'], ['die Kinder']), (['die Kinder'], ['der Chef'], ['ein Kuchen']), (['die Kinder'], ['ein Kuchen'], ['der Chef']), (['ein Kuchen'], ['der Chef'], ['die Kinder']), (['ein Kuchen'], ['die Kinder'], ['der Chef'])]


In [166]:
all_ungramm = ""
all_ungramm_list = [[] for _ in range(number_sentences)]
counter = 0
for stc_idx, stc in enumerate(tokens_wo_number): 
    for case_perm_idx, case_perm in enumerate(all_ungram_perm[stc_idx]):
        for case_idx, case in enumerate(all_ungram_perm[stc_idx][case_perm_idx]):
            principal_clause = f"{tokens_wo_number[stc_idx][0]} {tokens_wo_number[stc_idx][1]}, {tokens_wo_number[stc_idx][3]}"
            
            verb_arg = ""
            for vb_arg_idx, vb_arg in enumerate(all_ungram_perm[stc_idx][case_perm_idx][case_idx]):
                verb_arg += f"{vb_arg[0]} " 

            verb_subordinate = f"{tokens_wo_number[stc_idx][-3]} {tokens_wo_number[stc_idx][-2]}"
            all_ungramm += f"{principal_clause} {verb_arg}{verb_subordinate}.\n"
            all_ungramm_list[stc_idx].append(f"{principal_clause} {verb_arg}{verb_subordinate}.")
            counter += 1
            
            
counter = counter/(number_sentences)
        
print(len(all_ungramm))


157464


In [167]:
print(len(all_ungramm_list))
print(len(all_ungramm_list[0]))

20
108


In [168]:
print(counter)

108.0


In [169]:
print(all_ungramm)

Wir glauben, dass den Mann die Frauen ein Geschenk gegeben hat.
Wir glauben, dass den Mann ein Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen den Mann ein Geschenk gegeben hat.
Wir glauben, dass die Frauen ein Geschenk den Mann gegeben hat.
Wir glauben, dass ein Geschenk den Mann die Frauen gegeben hat.
Wir glauben, dass ein Geschenk die Frauen den Mann gegeben hat.
Wir glauben, dass der Mann die Frauen ein Geschenk gegeben hat.
Wir glauben, dass der Mann ein Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen der Mann ein Geschenk gegeben hat.
Wir glauben, dass die Frauen ein Geschenk der Mann gegeben hat.
Wir glauben, dass ein Geschenk der Mann die Frauen gegeben hat.
Wir glauben, dass ein Geschenk die Frauen der Mann gegeben hat.
Wir glauben, dass der Mann die Frauen ein Geschenk gegeben hat.
Wir glauben, dass der Mann ein Geschenk die Frauen gegeben hat.
Wir glauben, dass die Frauen der Mann ein Geschenk gegeben hat.
Wir glauben, dass die Frauen ein Geschen

### Check duplicates in grammatical and non grammatical datasets

#### Duplicates within grammatical dataset

In [17]:
duplicates_within_grams = []
where_duplicates_within_grams = []
for stc_idx, stc in enumerate(all_gramm_list):
    stc_counter = 0
    for sub_stc_idx, sub_stc in enumerate(all_gramm_list[stc_idx]):  
        counter = all_gramm_list[stc_idx].count(sub_stc)
        if counter > 1:
            where_duplicates = (stc_idx, sub_stc_idx)
            where_duplicates_within_grams.append(where_duplicates)
            stc_counter += 1
    duplicates_within_grams.append(stc_counter)
           
print(duplicates_within_grams)
#print(where_duplicates_within_grams)

NameError: name 'all_gramm_list' is not defined

In [171]:
# how many times are ungrammatical duplicates present in ungrammatical dataset
how_many_gram_duplicates = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(all_gramm_list):
    for sub_stc_idx, sub_stc in enumerate(all_gramm_list[stc_idx]):
        how_many_gram_duplicates[stc_idx].append(all_gramm_list[stc_idx].count(sub_stc))
print(len(how_many_gram_duplicates))
print(len(how_many_gram_duplicates[0]))
print(how_many_gram_duplicates)

20
36
[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1,

#### Duplicates between grammatical dataset and ungrammatical dataset

In [172]:
duplicates = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(all_ungramm_list):
    duplicates[stc_idx] = [stc for stc in all_gramm_list[stc_idx] if stc in all_ungramm_list[stc_idx]]

In [173]:
print(duplicates)

[['Wir glauben, dass der Mann die Frauen einem Geschenk gegeben hat.', 'Wir glauben, dass der Mann einem Geschenk die Frauen gegeben hat.', 'Wir glauben, dass die Frauen der Mann einem Geschenk gegeben hat.', 'Wir glauben, dass die Frauen einem Geschenk der Mann gegeben hat.', 'Wir glauben, dass einem Geschenk der Mann die Frauen gegeben hat.', 'Wir glauben, dass einem Geschenk die Frauen der Mann gegeben hat.', 'Wir glauben, dass der Mann den Frauen ein Geschenk gegeben hat.', 'Wir glauben, dass der Mann ein Geschenk den Frauen gegeben hat.', 'Wir glauben, dass den Frauen der Mann ein Geschenk gegeben hat.', 'Wir glauben, dass den Frauen ein Geschenk der Mann gegeben hat.', 'Wir glauben, dass ein Geschenk der Mann den Frauen gegeben hat.', 'Wir glauben, dass ein Geschenk den Frauen der Mann gegeben hat.', 'Wir glauben, dass den Mann die Frauen einem Geschenk gegeben hat.', 'Wir glauben, dass den Mann einem Geschenk die Frauen gegeben hat.', 'Wir glauben, dass die Frauen den Mann einem

In [174]:
# How many sentences of grammatical dataset are present in ungrammatical dataset per original sentence
duplicates_counter = []
for stc_idx, stc in enumerate(duplicates):
    counter = len(duplicates[stc_idx])
    duplicates_counter.append(counter)
    print(stc_idx, counter)

0 36
1 36
2 36
3 36
4 36
5 24
6 24
7 24
8 24
9 36
10 36
11 36
12 36
13 36
14 0
15 36
16 36
17 36
18 24
19 36


In [108]:
print(all_gramm_list[14][0])

Ich glaube, dass der Junge seinen Kumpel einem Kugelschreiber mitgebracht hat.


In [175]:
# how many times are grammatical sentences present in ungrammatical sentences
how_many_duplicates = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(duplicates):
    for sub_stc_idx, sub_stc in enumerate(duplicates[stc_idx]):
        how_many_duplicates[stc_idx].append(all_ungramm_list[stc_idx].count(sub_stc))

In [177]:
print(how_many_duplicates)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

#### Duplicates within ungrammatical dataset

In [178]:
duplicates_within_ungrams = []
where_duplicates_within_ungrams = []

nom_duplicates = 0
acc_duplicates = 0
dat_duplicates = 0 
total_counter = 0

for stc_idx, stc in enumerate(all_ungramm_list):
    stc_counter = 0
    for sub_stc_idx, sub_stc in enumerate(all_ungramm_list[stc_idx]):  
        counter = all_ungramm_list[stc_idx].count(sub_stc)
        if counter > 1:
            total_counter += 1
            where_duplicates = (stc_idx, sub_stc_idx)
            where_duplicates_within_ungrams.append(where_duplicates)
            stc_counter += 1
            
            if sub_stc_idx <= 35:
                nom_duplicates += 1
            elif 35 < sub_stc_idx <= 72: 
                acc_duplicates += 1
            elif sub_stc_idx > 72: 
                dat_duplicates =+ 1
                
    duplicates_within_ungrams.append(stc_counter)

nb_sentences_per_violation = number_sentences*len(all_ungramm_list[0])/3

print("duplicates in ungrammatical dataset:", total_counter)
print("nb duplicates per case violation:")
print("nominative:", nom_duplicates, ";", "accusative:", acc_duplicates, ";", "dative:", dat_duplicates)
print("proportion of duplicates per case violation:")
print("nominative:", nom_duplicates/nb_sentences_per_violation, ";", "accusative:", acc_duplicates/nb_sentences_per_violation, ";", "dative:", dat_duplicates/nb_sentences_per_violation)

duplicates in ungrammatical dataset: 1332
nb duplicates per case violation:
nominative: 444 ; accusative: 461 ; dative: 1
proportion of duplicates per case violation:
nominative: 0.6166666666666667 ; accusative: 0.6402777777777777 ; dative: 0.001388888888888889


In [179]:
print(duplicates_within_ungrams)
#print(where_duplicates_within_ungrams)

[72, 72, 108, 72, 72, 36, 36, 36, 36, 72, 108, 72, 108, 72, 0, 108, 72, 72, 36, 72]


In [181]:
# how many times are ungrammatical duplicates present in ungrammatical dataset
how_many_ungram_duplicates = [[] for _ in range(number_sentences)]
for stc_idx, stc in enumerate(all_ungramm_list):
    for sub_stc_idx, sub_stc in enumerate(all_ungramm_list[stc_idx]):
        how_many_ungram_duplicates[stc_idx].append(all_ungramm_list[stc_idx].count(sub_stc))
print(len(how_many_ungram_duplicates))
print(len(how_many_ungram_duplicates[0]))
print(how_many_ungram_duplicates)

20
108
[[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [3, 3, 3, 3, 3

In [184]:
print(how_many_ungram_duplicates[2])

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


### Save results

In [ ]:
BASE_PATH = "input_sentences/"
output_file = open(BASE_PATH + 'ungrammatical_sentences.txt', 'w')
output_file.write(all_ungramm)
output_file.close()

In [ ]:
def load_sentences(input_path):
    """
    Input: str, path to input file
    Output: loaded file
    """
    with open(input_path, "r") as f:
        output = f.read()

    return output

In [ ]:
with open("input_sentences/ungrammatical_sentences.txt", "r") as f:
     ungrammatical_sentences = f.read().split("\n")

In [ ]:
print(ungrammatical_sentences)

In [ ]:
def load_sentences(input_path):
    """
    Input: str, path to input file
    Output: loaded file
    """
    with open(input_path, "r") as f:
        output = f.read()

    return output

In [ ]:
path = "input_sentences/grammatical_sentences.txt"
grammatical_test_sentences = load_sentences(path)

In [ ]:
print(grammatical_test_sentences)

In [ ]:
def tokenizer(sentences, word_to_idx):
    """
    Input: loaded text file
    Output: tokenized text
    """
    sentences = sentences.replace(".", ". <eos>")  # adding end of sentence symbols for tokenization

    sentences = sentences.split("<eos>")  # separates sentences
    sentences.pop()  # removes last element (a whitespace)
    print("number of test sentences: ", len(sentences))

    for i in range(len(sentences)):  # tokenizes sentences
        sentences[i] = sentences[i].replace(",", " ,").replace(".", " .").replace(":", " :").replace("?", " ?").replace("!", " !").replace(";", " ;")
        sentences[i] = sentences[i].lower()
        sentences[i] = sentences[i].split()

    tokenized_sentences = [[] for _ in range(len(sentences))]
    for i in range(len(sentences)):
        for j in range(len(sentences[i])):
            word = sentences[i][j]
            tokenized_sentences[i].append(word_to_idx[word] if word in word_to_idx else 2)

    return tokenized_sentences